### Setting for Colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/Projects/kubig19th-conference-llm/ym

/content/drive/MyDrive/Projects/kubig19th-conference-llm/ym


In [3]:
!pip install python-dotenv semanticscholar langchain langchain_openai langchain_core langchainhub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00


In [4]:
!pip install feedparser PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6049 sha256=51c4531939e38d58e12e08223d7bf969556836dffacc5204c011171fef04de59
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k


### Import Modules

In [5]:
import warnings
warnings.filterwarnings('ignore')
import torch
import os
from pprint import pprint
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from dotenv import load_dotenv

from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_openai import ChatOpenAI

from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent

In [6]:
import semantic_scholoar_api as ss

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [7]:
dotenv_path = '.env'
load_dotenv(dotenv_path)
openai_api = os.getenv("OPENAI_API_KEY")
ss_api_key = os.getenv("SEMANTIC_SCHOLAR_API_KEY")

### Define Tools

In [ ]:
class load_paper_input(BaseModel):
    query: str = Field(description="target paper title")
    start_page: int = Field(description='start page number')

loadpaper = StructuredTool.from_function(
    func=ss.query2content,
    name="loadpaper",
    description="The `loadPaper` tool is designed to facilitate the process of retrieving and reading academic papers based on a given search query. The `query` parameter is a string representing the title of the paper. The `loadPaper` tool extracts and returns the text content from the PDF. If the paper does not contain the desired information, you can adjust the start_page argument to specify the starting page for reading. For instance, to extract the title, abstract, or detailed method sections, set start_page to 1. Conversely, for sections like the appendix that are typically found towards the end of the paper, set start_page to a higher number, such as 9.",
    args_schema=load_paper_input
)

In [ ]:
class recommend_reference_input(BaseModel):
    query: str = Field(description="target paper title")

recommend_reference = StructuredTool.from_function(
    func=ss.reference_recommend,
    name="recommend_reference",
    description="The reference_recommend function recommends relevant academic papers based on a given query, focusing on papers that the target paper's references. This tool is ideal for researchers and academics looking to find related literature that has been directly cited by the target paper.",
    args_schema=recommend_reference_input
)

In [ ]:
class citation_recommend_input(BaseModel):
    query: str = Field(description="target paper title")

recommend_citation = StructuredTool.from_function(
    func=ss.citation_recommend,
    name="recommend_citation",
    description="The recommend_citation function identifies and recommends subsequent papers **that have cited a given target paper**, providing valuable insights into the evolution and impact of the research. This tool helps researchers discover influential follow-up studies and stay updated with the latest developments in their field.",
    args_schema=citation_recommend_input
)

### Make Agent with GPT3.5

In [ ]:
model = ChatOpenAI(model="gpt-3.5-turbo")

In [ ]:
tools = [loadpaper, recommend_reference, recommend_citation]
# functions = [convert_to_openai_function(t) for t in tools]
# functions[1]

In [ ]:
# load Agent prompt
prompt = hub.pull("hwchase17/openai-tools-agent")

In [ ]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-tools-agent', 'lc_hub_commit_hash': 'c18672812789a3b9697656dd539edf0120285dcae36396d0b548ae42a4ed66f5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlacehold

In [ ]:
# Choose the LLM that will drive the agent
# Only certain models support this
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(llm, tools, prompt)
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
# If you want the model to see the specific page or specific content in the paper, mention the page in the prompt.

output = agent_executor.invoke({"input": "explain about the math expression about the Filtering api calls in the 3 page of the paper 'Language Models Can Teach Themselves to Use Tools'"})



> Entering new AgentExecutor chain...

Invoking: `loadpaper` with `{'query': 'Language Models Can Teach Themselves to Use Tools', 'start_page': 3}`


### Searched Paper ###
Title: Toolformer: Language Models Can Teach Themselves to Use Tools
Authors: Thomas Scialom
Published: 2023-02-09T16:49:57Z
ID: 2302.04761v1
File ./papers_db/2302.04761v1.pdf already exists. Skipping download.
Page limit reached at 4
Your task is to add calls to a Question Answering API to a piece of text. The questions should help you get information required to complete the text. You can call the API by writing "[QA(question)]" where "question" is the question you want to ask. Here are some examples of API calls: Input: Joe Biden was born in Scranton, Pennsylvania. Output: Joe Biden was born in [QA("Where was Joe Biden born?")] Scranton, [QA("In which state is Scranton?")] Pennsylvania. Input: Coca-Cola, or Coke, is a carbonated soft drink manufactured by the Coca-Cola Company. Output: Coca-Cola, or [QA("What o

In [ ]:
pprint(output['output'])

('The math expression related to the Filtering API Calls in the 3rd page of '
 'the paper "Language Models Can Teach Themselves to Use Tools" involves a '
 'process of filtering API calls based on a weighted cross entropy loss '
 'calculation. Here is a summary of the filtering process:\n'
 '\n'
 '1. **Sampling API Calls**:\n'
 '   - A prompt P(x) is used to encourage the language model (LM) to annotate '
 'an example x with API calls.\n'
 '   - The probability p_i is computed for each position i to start an API '
 'call.\n'
 '   - Positions with probabilities above a sampling threshold are selected, '
 'and up to m API calls are obtained for each selected position.\n'
 '\n'
 '2. **Executing API Calls**:\n'
 '   - All API calls generated by the LM are executed to obtain the '
 'corresponding results using various methods depending on the API.\n'
 '\n'
 '3. **Filtering API Calls**:\n'
 '   - The position of the API call c_i in the sequence x and the response r_i '
 'from the API are con

In [ ]:
output = agent_executor.invoke({"input": "논문 'Language Models Can Teach Themselves to Use Tools'와 비슷한 후속 논문을 추천해주고, 해당 논문의 abstract가 내가 준 논문과 어떤 차이가 있는지 알려줘"})
# TODO : we do not need paperID in this tool.

In [ ]:
pprint(output['output'])

('Here are some recommended papers related to "Language Models Can Teach '
 'Themselves to Use Tools":\n'
 '\n'
 '1. **Gorilla: Large Language Model Connected with Massive APIs**\n'
 '   - Abstract: Large Language Models (LLMs) have seen significant '
 'advancements recently, excelling in various tasks. Gorilla, a finetuned '
 'LLaMA-based model, surpasses the performance of GPT-4 in writing API calls. '
 'It demonstrates a strong capability to adapt to test-time document changes '
 'and mitigate hallucination issues.\n'
 '   - Publication Date: May 24, 2023\n'
 '\n'
 '2. **Mind2Web: Towards a Generalist Agent for the Web**\n'
 '   - Abstract: Introduces Mind2Web, a dataset for developing generalist '
 'agents for the web. It provides diverse tasks from real-world websites and '
 'explores using large language models for building generalist web agents.\n'
 '   - Publication Date: June 9, 2023\n'
 '\n'
 '3. **ToolLLM: Facilitating Large Language Models to Master 16000+ Real-world '
 'AP

In [ ]:
# TODO : Change Arxiv Search enging to Semantic Scholar
# TODO : error handling when the tool called.
# Because below code doesn't work. with the difference of 'Gorilla: '
# output = agent_executor.invoke({"input": "summary the abstract of the paper 'Gorilla: Large Language Model Connected with Massive APIs'"})

output = agent_executor.invoke({"input": "summary the abstract of the paper 'Large Language Model Connected with Massive APIs'"})



> Entering new AgentExecutor chain...

Invoking: `loadpaper` with `{'query': 'Large Language Model Connected with Massive APIs', 'start_page': 1}`


### Searched Paper ###
Title: Gorilla: Large Language Model Connected with Massive APIs
Authors: Joseph E. Gonzalez
Published: 2023-05-24T16:48:11Z
ID: 2305.15334v1
Page limit reached at 4
Gorilla: Large Language Model Connected with Massive APIs Shishir G. Patil1∗Tianjun Zhang1,∗Xin Wang2Joseph E. Gonzalez1 1UC Berkeley2Microsoft Research sgp@berkeley.edu Abstract Large Language Models (LLMs) have seen an impressive wave of advances re- cently, with models now excelling in a variety of tasks, such as mathematical reasoning and program synthesis. However, their potential to effectively use tools via API calls remains unfulfilled. This is a challenging task even for today’s state-of- the-art LLMs such as GPT-4, largely due to their inability to generate accurate input arguments and their tendency to hallucinate the wrong usage of an API c

In [ ]:
pprint(output['output'])

('The paper titled "Large Language Model Connected with Massive APIs" '
 'introduces Gorilla, a finetuned LLaMA-based model that surpasses the '
 'performance of GPT-4 in writing API calls. Gorilla, when combined with a '
 'document retriever, demonstrates the ability to adapt to test-time document '
 'changes, reducing hallucination errors commonly encountered with LLMs. The '
 'paper introduces APIBench, a dataset consisting of HuggingFace, TorchHub, '
 "and TensorHub APIs, to evaluate the model's ability accurately. Gorilla "
 'significantly outperforms GPT-4 in terms of API functionality accuracy and '
 'reduces hallucination errors. The paper emphasizes the importance of '
 'empowering LLMs to use tools via API calls to access vast knowledge bases '
 'and accomplish complex computational tasks effectively.')


In [ ]:
output = agent_executor.invoke({"input": "recommend the reference works of the paper 'Language Models Can Teach Themselves to Use Tools'"})



> Entering new AgentExecutor chain...

Invoking: `recommend_reference` with `{'query': 'Language Models Can Teach Themselves to Use Tools'}`


[{'paperId': '90abbc2cf38462b954ae1b772fac9532e2ccd8b0', 'title': 'Language Models are Few-Shot Learners', 'abstract': "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something which current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches. Specifically, we train GPT-3, an autore

In [ ]:
pprint(output['output'])

('Here are some recommended reference works related to the paper "Language '
 'Models Can Teach Themselves to Use Tools":\n'
 '\n'
 '1. **Paper Title:** Language Models are Few-Shot Learners\n'
 '   - **Abstract:** Recent work has demonstrated substantial gains on many '
 'NLP tasks and benchmarks by pre-training on a large corpus of text followed '
 'by fine-tuning on a specific task. This paper shows that scaling up language '
 'models greatly improves task-agnostic, few-shot performance, sometimes even '
 'reaching competitiveness with prior state-of-the-art fine-tuning '
 'approaches.\n'
 '   - **Publication Date:** 2020-05-28\n'
 '\n'
 '2. **Paper Title:** PaLM: Scaling Language Modeling with Pathways\n'
 '   - **Abstract:** This paper explores the impact of scale on few-shot '
 'learning by training a 540-billion parameter, densely activated, Transformer '
 'language model called Pathways Language Model PaLM. It achieves breakthrough '
 'performance on language understanding and 

### Todo : Make Agent with Llama

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
# we can use 262k version of Llama!
model_name = 'gradientai/Llama-3-8B-Instruct-262k'

dtype = "float16"
if torch.cuda.is_bf16_supported():
    dtype = "bfloat16"

quantization_config = BitsAndBytesConfig(load_in_4bit=True)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=getattr(torch, dtype), quantization_config=quantization_config,
                                             device_map="auto",  cache_dir=os.getenv("HF_HOME", "~/.cache/huggingface"))
model.eval()

tokenizer = AutoTokenizer.from_pretrained(model_name)

### HTML

In [8]:
!pip install beautifulsoup4

In [9]:
import requests
from bs4 import BeautifulSoup

In [15]:
class GetPaper:
    def __init__(self, ss_api_key):
        self.ss_api_key = ss_api_key

    def get_paper_info_by_title(self, title):
        """논문의 제목으로 정보를 가져오는 함수"""
        # Define the API endpoint URL
        url = 'https://api.semanticscholar.org/graph/v1/paper/search?query={}&fields=paperId,title,abstract,authors,citations,fieldsOfStudy,influentialCitationCount,isOpenAccess,openAccessPdf,publicationDate,publicationTypes,references,venue'

        headers = {'x-api-key': self.ss_api_key}
        response = requests.get(url.format(title), headers=headers).json()

        if response.get('data'):
            paper = response['data'][0]
            return paper
        else:
            return None

    def get_ar5iv_url(self, paper):
        "논문의 ar5iv 주소를 받아오는 함수"
        external_ids = paper.get('openAccessPdf', {})
        arxiv_id = external_ids.get('url')
        if 'http' in arxiv_id:
            arxiv_id = arxiv_id.split('/')[-1]
            return f"https://ar5iv.org/abs/{arxiv_id}"
        else:
            return None

    def get_html_from_url(self, url, sections:list=None):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        all_sections = soup.find_all('section')

        # h1부터 h6까지 태그 추출
        headers = soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])

        # 태그와 내용을 계층 구조로 저장
        header_list = [(header.name, header.text.strip()) for header in headers]

        for tag, text in header_list:
            level = int(tag[1])  # 태그에서 레벨을 추출 (h1 -> 1, h2 -> 2, ...)
            print('  ' * (level - 1) + text)

        return header_list

    def extract_text_under_headers(url, text_list):
        # 페이지 요청 및 읽기
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        # 결과를 저장할 변수
        results = []

        # 텍스트 리스트를 순회하며 각 텍스트에 해당하는 헤더와 그 아래의 텍스트를 추출
        for text in text_list:
            header_tag = soup.find(lambda tag: tag.name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6'] and text in tag.get_text(strip=True))
            if header_tag:
                header_text = header_tag.get_text(strip=True)
                header_level = int(header_tag.name[1])
                current_header = {'tag': header_tag.name, 'text': header_text, 'subsections': []}
                results.append(current_header)

                next_element = header_tag.find_next_sibling()
                while next_element:
                    if next_element.name and next_element.name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
                        next_level = int(next_element.name[1])
                        if next_level <= header_level:
                            break

                        # If it's a tag and within our header range
                        if next_element.name and next_element.name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
                            next_text = next_element.get_text(strip=True)
                            next_subheader = {'tag': next_element.name, 'text': next_text, 'subsections': []}
                            current_header['subsections'].append(next_subheader)
                            current_header = next_subheader
                            header_level = next_level
                    else:
                        if 'subsections' not in current_header:
                            current_header['subsections'] = []
                        current_header['subsections'].append({'tag': 'p', 'text': next_element.get_text(strip=True)})

                    next_element = next_element.find_next_sibling()

        return results

    def print_hierarchy(self, hierarchy, level=0):
        for section in hierarchy:
            print('  ' * level + f"{section['text']}")
            if 'subsections' in section:
                self.print_hierarchy(section['subsections'], level + 1)


    def search_paper_by_title(self, title):
        '''
        INPUT : title of the paper
        OUTPUT : list of sections in paper
        '''
        paper = self.get_paper_info_by_title(title)
        url = self.get_ar5iv_url(paper)
        # todo : Error Handling when no ar5iv URL.
        # arxiv pdf를 다운받고, pdf읽는 식으로 handle가능.
        if (url == None):
            raise "NO ar5iv URL is founded"
        html_headers = self.get_html_from_url(url)
        return html_headers

    def load_paper_by_sections(self, title, sections):
        '''
        INPUT : title of paper,
                list of sections in paper
        OUTPUT : text of the paper
        '''
        paper = self.get_paper_info_by_title(title)
        url = self.get_ar5iv_url(paper)
        text = self.extract_text_under_headers(url, sections)
        return text


In [16]:
paperModule = GetPaper(ss_api_key)

In [17]:
text = paperModule.load_paper_by_sections(title = 'Large Language Model Connected with Massive APIs', sections=['Methodology', 'Conclusion'])

In [18]:
text

[{'tag': 'h2',
  'text': '3Methodology',
  'subsections': [{'tag': 'p',
    'text': 'Figure 3:Gorilla: A system for enabling LLMs to interact with APIs.The upper half represents the training procedure as described in Sec3. This is the most exhaustive API data-set for ML to the best of our knowledge. During inference (lower half), Gorilla supports two modes - with retrieval, and zero-shot. In this example, it is able to suggest the right API call for generating the image from the user’s natural language query.'},
   {'tag': 'p',
    'text': 'In this section, we describe APIBench, a comprehensive benchmark constructed from TorchHub, TensorHub, and HuggingFace API Model Cards. We begin by outlining the process of collecting the API dataset and how we generated instruction-answer pairs. We then introduce Gorilla, a novel training paradigm with a information–retriever incorporated into the training and inference pipelines. Finally, we present our AST tree matching evaluation metric.'},
   {

In [ ]:
header = paperModule.search_paper_by_title(title)
content = paperModule.extract_text_under_headers(url, text_list)

Gorilla: Large Language Model Connected with Massive APIs
          Abstract
  1 Introduction
  2 Related Work
        Large Language Models
        Tool Usage
        LLMs for Program Synthesis
  3 Methodology
    3.1 Dataset Collection
        API Documentation
        Instruction Generation
    3.2 Gorilla
        API Call with Constraints
        Retriever-Aware training
        Gorilla Inference
    3.3 Verifying APIs
        AST Sub-Tree Matching
  4 Evaluation
        Baselines
        Retrievers
    4.1 AST Accuracy on API call
        Finetuning without Retrieval
        Finetuning with Retrieval
        Hallucination with LLM
    4.2 Test-Time Documentation Change
    4.3 API Call with Constraints
  5 Conclusion
  6 Limitations & Social Impacts
  7 Acknowledgement
  References
  8 Appendix
    8.1 Dataset Details
        Domain Classification
        API Call Task
        API Provider Component
        Explanation Element
        Code
    8.2 Gorilla Details
        Data
    

In [ ]:
content

[{'tag': 'h1',
  'text': 'Gorilla: Large Language Model Connected with Massive APIs',
  'subsections': [{'tag': 'p',
    'text': 'Shishir G. Patil1Tianjun Zhang1,∗Xin Wang2Joseph E. Gonzalez11UC Berkeley2Microsoft Researchsgp@berkeley.eduEqual contribution.'},
   {'tag': 'p',
    'text': 'AbstractLarge Language Models (LLMs) have seen an impressive wave of advances recently, with models now excelling in a variety of tasks, such as mathematical reasoning and program synthesis. However, their potential to effectively use tools via API calls remains unfulfilled. This is a challenging task even for today’s state-of-the-art LLMs such as GPT-4, largely due to their inability to generate accurate input arguments and their tendency to hallucinate the wrong usage of an API call. We release Gorilla, a finetuned LLaMA-based model that surpasses the performance of GPT-4 on writing API calls. When combined with a document retriever, Gorilla demonstrates a strong capability to adapt to test-time doc

In [ ]:
# Example usage
title = "Gorilla: Large Language Model Connected with Massive APIs"
paper = get_paper_info_by_title(title, api_key)

if paper:
    abstract = paper.get('abstract', '')
    print(f"Title: {paper['title']}")
    print(f"Abstract: {abstract}")

    ar5iv_url = get_ar5iv_url(paper)
    if ar5iv_url:
        print(f"ar5iv URL: {ar5iv_url}")

x = get_html_from_url(ar5iv_url)

Title: Gorilla: Large Language Model Connected with Massive APIs
Abstract: Large Language Models (LLMs) have seen an impressive wave of advances recently, with models now excelling in a variety of tasks, such as mathematical reasoning and program synthesis. However, their potential to effectively use tools via API calls remains unfulfilled. This is a challenging task even for today's state-of-the-art LLMs such as GPT-4, largely due to their inability to generate accurate input arguments and their tendency to hallucinate the wrong usage of an API call. We release Gorilla, a finetuned LLaMA-based model that surpasses the performance of GPT-4 on writing API calls. When combined with a document retriever, Gorilla demonstrates a strong capability to adapt to test-time document changes, enabling flexible user updates or version changes. It also substantially mitigates the issue of hallucination, commonly encountered when prompting LLMs directly. To evaluate the model's ability, we introduce 

In [ ]:

# 주어진 텍스트 리스트 예시
text_list = [
    'Gorilla: Large Language Model Connected with Massive APIs',
    'Introduction',
    'Dataset Collection',
    'Related Work'
]

# 함수 호출
extracted_texts = extract_text_under_headers(ar5iv_url, text_list)

# 결과 출력
for tag, header, text in extracted_texts:
    print(f"{tag}: {header}\n{text}\n\n\n\n")


tag: text
subsections




tag: text
subsections




tag: text
subsections




tag: text
subsections






In [ ]:
extracted_texts = extract_text_under_headers(ar5iv_url, text_list)

# 결과 출력
print_hierarchy(extracted_texts)

Gorilla: Large Language Model Connected with Massive APIs
  Shishir G. Patil1Tianjun Zhang1,∗Xin Wang2Joseph E. Gonzalez11UC Berkeley2Microsoft Researchsgp@berkeley.eduEqual contribution.
  AbstractLarge Language Models (LLMs) have seen an impressive wave of advances recently, with models now excelling in a variety of tasks, such as mathematical reasoning and program synthesis. However, their potential to effectively use tools via API calls remains unfulfilled. This is a challenging task even for today’s state-of-the-art LLMs such as GPT-4, largely due to their inability to generate accurate input arguments and their tendency to hallucinate the wrong usage of an API call. We release Gorilla, a finetuned LLaMA-based model that surpasses the performance of GPT-4 on writing API calls. When combined with a document retriever, Gorilla demonstrates a strong capability to adapt to test-time document changes, enabling flexible user updates or version changes. It also substantially mitigates th